In [2]:
import pandas as pd
import numpy as np
from sqlalchemy.engine import URL, create_engine, Inspector

# Use your own server connection here
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=DAIYAAN;"
            "Database=AIFMRM_ERS;"
            "Trusted_Connection=yes;")

cnxn_url = URL.create("mssql+pyodbc", query={"odbc_connect": cnxn_str})

engine = create_engine(cnxn_url)


# initialise connection via context manager           
with engine.connect() as cnxn:
        tables_df = pd.read_sql('SELECT [name] AS [table_name] FROM sys.tables', cnxn)
        table_name_list = tables_df.table_name
        select_template = 'SELECT * FROM {table_name}'
        # Dictionary of table names and their respective SQL queries
        frames_dict = {}
        for tname in table_name_list:
                query = select_template.format(table_name = tname)
                frames_dict[tname] = pd.read_sql(query, cnxn)

''''
tbl_Industry_Classification_Benchmark
tbl_FTSEJSE_Index_Series 
tbl_EOD_Equity_Data
tbl_EOD_Interest_Rate_Data 
tbl_Beta_Output 
tbl_BA_Beta_Output
tbl_Index_Constituents
'''

"'\ntbl_Industry_Classification_Benchmark\ntbl_FTSEJSE_Index_Series \ntbl_EOD_Equity_Data\ntbl_EOD_Interest_Rate_Data \ntbl_Beta_Output \ntbl_BA_Beta_Output\ntbl_Index_Constituents\n"

In [11]:
tbl_Industry_Classification_Benchmark = frames_dict["tbl_Industry_Classification_Benchmark"]
tbl_Index_Constituents = frames_dict["tbl_Index_Constituents"]
tbl_BA_Beta_Output = frames_dict["tbl_BA_Beta_Output"]
tbl_FTSEJSE_Index_Series = frames_dict["tbl_FTSEJSE_Index_Series"]
tbl_EOD_Equity_Data = frames_dict["tbl_EOD_Equity_Data"]
tbl_EOD_Interest_Rate_Data = frames_dict["tbl_EOD_Interest_Rate_Data"]
tbl_Beta_Output = frames_dict["tbl_Beta_Output"]

'''
0001 - Oil & Gas
1000 - Basic Materials
2000 - Industrials
3000 - Consumer Goods
4000 - Health Care
5000 - Consumer Services
6000 - Telecommunications
7000 - Utilities
8000 - Financials
9000 - Technology
'''

'\n0001 - Oil & Gas\n1000 - Basic Materials\n2000 - Industrials\n3000 - Consumer Goods\n4000 - Health Care\n5000 - Consumer Services\n6000 - Telecommunications\n7000 - Utilities\n8000 - Financials\n9000 - Technology\n'

## getICsAndWeights() function

In [18]:
def GetICsAndWeights(rDate,IndexCode,dbo_tbl_Index_Constituents):
    
    #rDate will be supplied by the user: consisting of year and Quarter 
    rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m")
    rDate_Month = rDate.month
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = dbo_tbl_Index_Constituents["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Month = Dates_Col.month
    Dates_Col_Year = Dates_Col.year

    #Filter tbl_Index_Constituents using supplied quarter and year data from rData
    tbl_Index_Constituents_Date = dbo_tbl_Index_Constituents.loc[(Dates_Col_Month == rDate_Month) & (Dates_Col_Year == rDate_Year),]


    #IndexCode is provided by user: "ALSI", "FLED", "LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY" or "ALTI"
    IndexCode = IndexCode #provided as input by the user. "ALSI" for testing purposes

    #function to identify The index column that must be searched
    def Index_Col_Identifier(argument):
        match argument:
            case "FLED":
                return "ALSI New"
            case  "LRGG"|"MIDC"|"SMILC":
                return "Index New"
            case default:
                return argument+" New"
    
    IndexCode_Col = Index_Col_Identifier(IndexCode) #Obtain column name to search relevant rows

    #Filter tbl_Index_Constituents_Date using supplied IndexCode in the column identified from Index_Col_Identifier function
    tbl_Index_Constituents_final = tbl_Index_Constituents_Date[tbl_Index_Constituents_Date[IndexCode_Col] == IndexCode]

    #Generate results table with Shares and corresponding share weights
    Alpha = pd.DataFrame(tbl_Index_Constituents_final.loc[:,"Alpha"])
    Gross_Market_Capitalisation = np.array(tbl_Index_Constituents_final.loc[:,"Gross Market Capitalisation"])
    Weigths = pd.DataFrame(Gross_Market_Capitalisation/np.sum(Gross_Market_Capitalisation))
    Gross_Market_Capitalisation = pd.DataFrame(Gross_Market_Capitalisation)
    ICB_SubSector = pd.DataFrame(tbl_Index_Constituents_final.loc[:,"ICB Sub-Sector"])
    Results = pd.concat([Alpha.reset_index(drop=True), Weigths.reset_index(drop=True),Gross_Market_Capitalisation.reset_index(drop=True),ICB_SubSector.reset_index(drop=True)],axis=1)
    Results.columns = ['Alpha','Weights','Gross Market Capitalisation','ICB Sub-Sector']

    return Results


Quarter_month = {1:3, 2:6, 3:9, 4:12}

rDate_year = '2019' #Get year from user
rDate_quarter = 2 #get quarter from user
rDate_month = str(Quarter_month[rDate_quarter])
rDate = rDate_year +"-"+ rDate_month #Create single date value from supplied year and quarter

IndexCode = "ALSI" #Get input from user
dbo_tbl_Index_Constituents = "tbl_Index_Constituents" #Get input from user
dbo_tbl_Index_Constituents = frames_dict[dbo_tbl_Index_Constituents]
Output1 = GetICsAndWeights(rDate,IndexCode,dbo_tbl_Index_Constituents)

## Function 2

In [24]:
def GetBetasMktAndSpecVols(rDate,ICs,tbl_BA_Beta_Output,mktIndexCode):

    #rDate will be supplied by the user: consisting of year and Quarter 
    rDate = pd.to_datetime(rDate, format = "%Y-%m")
    rDate_Month = rDate.month
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = tbl_BA_Beta_Output["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Month = Dates_Col.month
    Dates_Col_Year = Dates_Col.year

    #Filter tbl_BA_Beta_Output using supplied quarter and year data from rData
    tbl_BA_Beta_Output_Date = tbl_BA_Beta_Output.loc[(Dates_Col_Month == rDate_Month) & (Dates_Col_Year == rDate_Year),]

    #Market index code provided by the user which could be "J203", "J200", "J250", "J257" or "J258"
    mktIndexCode = mktIndexCode

    #Filter tbl_BA_Beta_Output_Dates using provided mktIndexCode
    tbl_BA_Beta_Output_mktIndex = tbl_BA_Beta_Output_Date.loc[tbl_BA_Beta_Output_Date["Index"] == mktIndexCode]


    #list of IndexCodes obtain from  the Alpha column in the Output of Function 1.
    ICs = ICs
    tbl_BA_Beta_Output_IC = tbl_BA_Beta_Output_mktIndex.loc[tbl_BA_Beta_Output_mktIndex["Instrument"].isin(ICs)]

    mktVol_row = tbl_BA_Beta_Output_mktIndex.loc[tbl_BA_Beta_Output_mktIndex["Instrument"] == mktIndexCode]


    #Generate results table with Shares and corresponding share weights
    Betas = tbl_BA_Beta_Output_IC.loc[:,"Beta"]
    specVols = tbl_BA_Beta_Output_IC.loc[:,"Unique Risk"]
    mktVol = mktVol_row.loc[:,"Total Risk"]
    Results = pd.concat([Betas.reset_index(drop=True), specVols.reset_index(drop=True),mktVol.reset_index(drop=True)],axis=1)
    Results.columns = ['Betas','specVols','mktVol']

    return Results


ICs = Output1["Alpha"]
dbo_tbl_BA_Beta_Output = "tbl_BA_Beta_Output"
dbo_tbl_BA_Beta_Output = frames_dict[dbo_tbl_BA_Beta_Output]
mktIndexCode = "J203"
Output2 = GetBetasMktAndSpecVols(rDate,ICs,dbo_tbl_BA_Beta_Output,mktIndexCode)



## Function 3

In [29]:

weights = np.transpose(np.matrix(Output1["Weights"]))
betas = np.transpose(np.matrix(Output2["Betas"]))
specVols = np.transpose(np.matrix(Output2["specVols"]))
mktVol = Output2["mktVol"][0]

def CalcStats(weights,betas,specVols,mktVol):

    pfBeta = round(np.matmul(np.transpose(weights),betas),3)

    sysCov = np.matmul(betas,np.transpose(betas))*(mktVol**2)

    pfSysVol = np.transpose(weights)@betas@np.transpose(betas)@weights*(mktVol**2)

    specCov = np.diagflat(specVols)@np.diagflat(specVols)

    pfSpecVol = np.transpose(weights)@(specCov)@weights

    totCov = sysCov + specCov

    pfVol = pfSysVol + pfSpecVol

    d = np.diagflat(np.matrix(np.diag(totCov)))
    
    corrMat = (np.linalg.inv(d)@(sysCov+specCov))@np.linalg.inv(d)

    Output = {"pfBeta" : pfBeta, "sysCov" : sysCov, "pfSysVol" : pfSysVol, "specCov" : specCov, "pfSpecVol" : pfSpecVol, "totCov" : totCov, "pfVol" : pfVol, "corrMat": corrMat}
    
    return (Output)






## Table Generating Practice

In [30]:
'''
0001 - Oil & Gas
1000 - Basic Materials
2000 - Industrials
3000 - Consumer Goods
4000 - Health Care
5000 - Consumer Services
6000 - Telecommunications
7000 - Utilities
8000 - Financials
9000 - Technology
'''

from numpy import average, diagflat


Quarter_month = {1:3, 2:6, 3:9, 4:12}

Date_1_year = '2018'
Date_1_quarter = 2
Date_1_month = str(Quarter_month[Date_1_quarter])
Date_2_year = '2020'
Date_2_quarter = 4
Date_2_month = str(Quarter_month[Date_2_quarter])

Date_1 = Date_1_year +" "+ Date_1_month
Date_2 = Date_2_year +" "+ Date_2_month

Date_1 = pd.to_datetime(Date_1, format = "%Y-%m")
Date_2 = pd.to_datetime(Date_2, format = "%Y-%m")

Dates = (tbl_Index_Constituents.loc[(tbl_Index_Constituents["Date"] >= Date_1) & (tbl_Index_Constituents["Date"] <= Date_2)]).loc[:,"Date"].unique() #get list of quarters between two dates


Industry_codes = tbl_Industry_Classification_Benchmark.loc[:,"Industry Code"].unique() #find all the industry codes
Industry_names = tbl_Industry_Classification_Benchmark.loc[:,"Industry"].unique()
Subsector = (tbl_Industry_Classification_Benchmark.loc[tbl_Industry_Classification_Benchmark["Industry Code"] == Industry_codes[5]]).loc[:,"Sub-Sector Code"].unique() #Find subsector codes belonging to index code selected

#pre-filtering database to only have rows belonging to listed subsector codes
#Temp = dbo_tbl_Index_Constituents.loc[dbo_tbl_Index_Constituents["ICB Sub-Sector"].isin(Subsector)] 
IndexCode_temp = "TOPI" #Select index code - Provided by the user

#run through first function
Results_1 = GetICsAndWeights(Date_1,IndexCode_temp,tbl_Index_Constituents)
Results_1_filtered = Results_1.loc[Results_1["ICB Sub-Sector"].isin(Subsector)] 

if len(Results_1_filtered)>0:
    Filtered_GMC = np.array(Results_1_filtered.loc[:,"Gross Market Capitalisation"])
    Total_GMC = np.array(Results_1.loc[:,"Gross Market Capitalisation"])

    Industry_Weights = pd.DataFrame((Filtered_GMC/np.sum(Total_GMC)))
    Industry_Weight_tot = "{:.2%}".format(float(np.sum(Industry_Weights)))


    ICs_temp = Results_1_filtered["Alpha"]
    mktIndexCode_temp = "J203"
    Results_2 = GetBetasMktAndSpecVols(Date_1,ICs_temp,tbl_BA_Beta_Output,mktIndexCode_temp)


    #A = average(Results_2["Betas"])
    #print(Results_2["Betas"])
    #print(A)
    betas_temp = Results_2["Betas"]
    specVols_temp = Results_2["specVols"]
    mktVol_temp = Results_2["mktVol"][0]
    

    betas_temp = np.transpose(np.matrix(Results_2["Betas"]))
    specVols_temp = np.transpose(np.matrix(Results_2["specVols"]))

    #Results_3 = CalcStats(Industry_Weights,betas_temp,specVols_temp,mktVol_temp)
    #print(Results_3)


    #Function 3
    pfBeta = np.matmul(np.transpose(Industry_Weights),betas_temp)

    sysCov = np.matmul(betas_temp,np.transpose(betas_temp))*(mktVol_temp**2)
    

    pfSysVol = np.transpose(Industry_Weights)@betas_temp@np.transpose(betas_temp)@Industry_Weights*(mktVol_temp**2)
    #pfSysVol = "{:.2%}".format(float(pfSysVol[0]))
    

    specCov = np.diagflat(specVols_temp)@np.diagflat(specVols_temp)
    

    pfSpecVol = np.transpose(Industry_Weights)@(specCov)@Industry_Weights
    #pfSpecVol = "{:.2%}".format(float(pfSpecVol[0]))
    

    totCov = sysCov + specCov

    

    pfVol = pfSysVol + pfSpecVol
   
    

    #d = np.transpose(np.matrix(np.diag(totCov)))
    d = np.diagflat(np.matrix(np.diag(totCov)))
    print(np.shape(d))
    print(d)
    corrMat = (np.linalg.inv(d)@(sysCov+specCov))@np.linalg.inv(d)
    #CorrMat = np.matmul((np.matmul(np.linalg.inv(D),(sysCov + specCov))),np.linalg.inv(D))

    #print(corrMat)

    #Results_3 = CalcStats(Industry_Weights,betas_temp,specVols_temp,mktVol_temp)


#get results from F1 that will be used as input for F2


'''
ICs_temp = Results_1["Alpha"]
mktIndexCode_temp = "J250"
#dbo_tbl_BA_Beta_Output = frames_dict[dbo_tbl_BA_Beta_Output]
Results_2 = GetBetasMktAndSpecVols(Date_1,ICs_temp,dbo_tbl_BA_Beta_Output,mktIndexCode_temp)

#Use results from F1 and F2 as inputs for F3
weights_temp = np.transpose(np.matrix(Results_1["Weights"]))
betas_temp = np.transpose(np.matrix(Results_2["Betas"]))
specVols_temp = np.transpose(np.matrix(Results_2["specVols"]))
mktVol_temp = Results_2["mktVol"][0]

Results_3 = CalcStats(weights_temp,betas_temp,specVols_temp,mktVol_temp)
'''



(9, 9)
[[0.05789394 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.0433039  0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.10275756 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.07719124 0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.06780551 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.05009061
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.10391905 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.07794079 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.06152985]]


'\nICs_temp = Results_1["Alpha"]\nmktIndexCode_temp = "J250"\n#dbo_tbl_BA_Beta_Output = frames_dict[dbo_tbl_BA_Beta_Output]\nResults_2 = GetBetasMktAndSpecVols(Date_1,ICs_temp,dbo_tbl_BA_Beta_Output,mktIndexCode_temp)\n\n#Use results from F1 and F2 as inputs for F3\nweights_temp = np.transpose(np.matrix(Results_1["Weights"]))\nbetas_temp = np.transpose(np.matrix(Results_2["Betas"]))\nspecVols_temp = np.transpose(np.matrix(Results_2["specVols"]))\nmktVol_temp = Results_2["mktVol"][0]\n\nResults_3 = CalcStats(weights_temp,betas_temp,specVols_temp,mktVol_temp)\n'

In [31]:

from unittest import result

''' 
#rDate will be supplied by the user: consisting of year and Quarter 
     rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m")
    rDate_Month = rDate.month
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = dbo_tbl_Index_Constituents["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Month = Dates_Col.month
    Dates_Col_Year = Dates_Col.year

 tbl_Index_Constituents_Date = dbo_tbl_Index_Constituents.loc[(Dates_Col_Month == rDate_Month) & (Dates_Col_Year == rDate_Year),]

'''
Quarter_month = {1:3, 2:6, 3:9, 4:12}

Date_1_year = '2018'
Date_1_quarter = 1
Date_1_month = str(Quarter_month[Date_1_quarter])

Date_2_year = '2018'
Date_2_quarter = 4
Date_2_month = str(Quarter_month[Date_2_quarter])

Date_1 = Date_1_year +"-"+ Date_1_month+"-01"
Date_2 = Date_2_year +"-"+ Date_2_month+"-28"

Date_1 = pd.to_datetime(Date_1, format = "%Y-%m-%d")
Date_2 = pd.to_datetime(Date_2, format = "%Y-%m-%d")

Dates = (tbl_Index_Constituents.loc[(tbl_Index_Constituents["Date"] >= Date_1) & (tbl_Index_Constituents["Date"] <= Date_2)]).loc[:,"Date"].unique() #get list of quarters between two dates
Industry_codes = tbl_Industry_Classification_Benchmark.loc[:,"Industry Code"].unique() #find all the industry codes
Industry_names = tbl_Industry_Classification_Benchmark.loc[:,"Industry"].unique()
IndexCode_temp = "TOPI"

#IndexCode is provided by user: "ALSI", "FLED", "LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY" or "ALTI"

df_Storage = pd.DataFrame(columns=['Date', 'Industry', 'Weight', 'Beta', 'SysVol', 'SpecVar'])
#put in If Else statement for conditions where the weights is 0 which will give error
it = 0
for i in Dates:
    for t in range(len(Industry_codes)):
        it = it + 1
        print("iteration:",it)
        print("Date:",i)
        
        Subsector = (tbl_Industry_Classification_Benchmark.loc[tbl_Industry_Classification_Benchmark["Industry Code"] == Industry_codes[t]]).loc[:,"Sub-Sector Code"].unique()
         #Select index code - Provided by the user

        #run through first function
        Results_1 = GetICsAndWeights(i,IndexCode_temp,tbl_Index_Constituents)
        Results_1_filtered = Results_1.loc[Results_1["ICB Sub-Sector"].isin(Subsector)] 

        #Place if-Else statement here:
        if len(Results_1_filtered)>0:

            Filtered_GMC = np.array(Results_1_filtered.loc[:,"Gross Market Capitalisation"])
            Total_GMC = np.array(Results_1.loc[:,"Gross Market Capitalisation"])

            Industry_Weights = pd.DataFrame((Filtered_GMC/np.sum(Total_GMC)))
            Industry_Weight_tot = "{:.2%}".format(float(np.sum(Industry_Weights)))

            ICs_temp = Results_1_filtered["Alpha"]
            mktIndexCode_temp = "J203"
            Results_2 = GetBetasMktAndSpecVols(i,ICs_temp,tbl_BA_Beta_Output,mktIndexCode_temp)

            mktVol_temp = Results_2["mktVol"][0]
            betas_temp = np.transpose(np.matrix(Results_2["Betas"]))
            specVols_temp = np.transpose(np.matrix(Results_2["specVols"]))

            Results_3 = CalcStats(Industry_Weights,betas_temp,specVols_temp,mktVol_temp)

            beta_output = Results_3["pfBeta"][0]
            SysVol_output = "{:.3%}".format(float(Results_3["pfSysVol"][0]))
            SpecVar_output = round(float(Results_3["pfSpecVol"][0]),6)
            
            date_output = pd.to_datetime(i)
            date_output = str(date_output.year)+"-Q"+str(date_output.quarter)


            df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
            print("Industry:", Industry_names[t])
        else:
            weight_output = "{:.2%}".format(0.000)
            date_output = pd.to_datetime(i)
            date_output = str(date_output.year)+"-Q"+str(date_output.quarter)
            df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': weight_output, 'Beta' : "     -", 'SysVol' : "     -", 'SpecVar' : "     -"}, ignore_index=True)
            print("Industry:", Industry_names[t])






iteration: 1
Date: 2018-03-19T00:00:00.000000000
Industry: Oil & Gas
iteration: 2
Date: 2018-03-19T00:00:00.000000000
Industry: Basic Materials
iteration: 3
Date: 2018-03-19T00:00:00.000000000
Industry: Industrials
iteration: 4
Date: 2018-03-19T00:00:00.000000000
Industry: Consumer Goods
iteration: 5
Date: 2018-03-19T00:00:00.000000000
Industry: Health Care
iteration: 6
Date: 2018-03-19T00:00:00.000000000
Industry: Consumer Services
iteration: 7
Date: 2018-03-19T00:00:00.000000000
Industry: Telecommunications
iteration: 8
Date: 2018-03-19T00:00:00.000000000
Industry: Utilities
iteration: 9
Date: 2018-03-19T00:00:00.000000000


C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': weight_output, 'Beta' : "     -", 'SysVol' : "     -", 'SpecVar' : "     -"}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

Industry: Financials
iteration: 10
Date: 2018-03-19T00:00:00.000000000
Industry: Technology
iteration: 11
Date: 2018-06-18T00:00:00.000000000
Industry: Oil & Gas
iteration: 12
Date: 2018-06-18T00:00:00.000000000
Industry: Basic Materials
iteration: 13
Date: 2018-06-18T00:00:00.000000000
Industry: Industrials
iteration: 14
Date: 2018-06-18T00:00:00.000000000
Industry: Consumer Goods
iteration: 15
Date: 2018-06-18T00:00:00.000000000


C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': weight_output, 'Beta' : "     -", 'SysVol' : "     -", 'SpecVar' : "     -"}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

Industry: Health Care
iteration: 16
Date: 2018-06-18T00:00:00.000000000
Industry: Consumer Services
iteration: 17
Date: 2018-06-18T00:00:00.000000000
Industry: Telecommunications
iteration: 18
Date: 2018-06-18T00:00:00.000000000
Industry: Utilities
iteration: 19
Date: 2018-06-18T00:00:00.000000000
Industry: Financials
iteration: 20
Date: 2018-06-18T00:00:00.000000000
Industry: Technology
iteration: 21
Date: 2018-09-25T00:00:00.000000000
Industry: Oil & Gas
iteration: 22
Date: 2018-09-25T00:00:00.000000000
Industry: Basic Materials
iteration: 23
Date: 2018-09-25T00:00:00.000000000
Industry: Industrials
iteration: 24
Date: 2018-09-25T00:00:00.000000000
Industry: Consumer Goods
iteration: 25
Date: 2018-09-25T00:00:00.000000000
Industry: Health Care
iteration: 26
Date: 2018-09-25T00:00:00.000000000
Industry: Consumer Services
iteration: 27
Date: 2018-09-25T00:00:00.000000000
Industry: Telecommunications
iteration: 28
Date: 2018-09-25T00:00:00.000000000


C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be

Industry: Utilities
iteration: 29
Date: 2018-09-25T00:00:00.000000000
Industry: Financials
iteration: 30
Date: 2018-09-25T00:00:00.000000000
Industry: Technology
iteration: 31
Date: 2018-12-24T00:00:00.000000000
Industry: Oil & Gas
iteration: 32
Date: 2018-12-24T00:00:00.000000000
Industry: Basic Materials
iteration: 33
Date: 2018-12-24T00:00:00.000000000
Industry: Industrials
iteration: 34
Date: 2018-12-24T00:00:00.000000000
Industry: Consumer Goods
iteration: 35
Date: 2018-12-24T00:00:00.000000000
Industry: Health Care
iteration: 36
Date: 2018-12-24T00:00:00.000000000
Industry: Consumer Services
iteration: 37
Date: 2018-12-24T00:00:00.000000000
Industry: Telecommunications
iteration: 38
Date: 2018-12-24T00:00:00.000000000
Industry: Utilities
iteration: 39
Date: 2018-12-24T00:00:00.000000000
Industry: Financials
iteration: 40
Date: 2018-12-24T00:00:00.000000000
Industry: Technology


C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1615535336.py:91: FutureWarning: The frame.append method is deprecated and will be

## Synthetic Tab Function

In [33]:
def SyntheticTable(Date_1_year,Date_1_quarter,Date_2_year,Date_2_quarter,IndexCode_temp):
    Quarter_month = {1:3, 2:6, 3:9, 4:12}
    Date_1_month = str(Quarter_month[Date_1_quarter])
    Date_2_month = str(Quarter_month[Date_2_quarter])

    Date_1 = Date_1_year +"-"+ Date_1_month+"-01"
    Date_2 = Date_2_year +"-"+ Date_2_month+"-28"

    Date_1 = pd.to_datetime(Date_1, format = "%Y-%m-%d")
    Date_2 = pd.to_datetime(Date_2, format = "%Y-%m-%d")

    Dates = (tbl_Index_Constituents.loc[(tbl_Index_Constituents["Date"] >= Date_1) & (tbl_Index_Constituents["Date"] <= Date_2)]).loc[:,"Date"].unique() #get list of quarters between two dates
    Industry_codes = tbl_Industry_Classification_Benchmark.loc[:,"Industry Code"].unique() #find all the industry codes
    Industry_names = tbl_Industry_Classification_Benchmark.loc[:,"Industry"].unique()

    #IndexCode is provided by user: "ALSI", "FLED", "LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY" or "ALTI"

    df_Storage = pd.DataFrame(columns=['Date', 'Industry', 'Weight', 'Beta', 'SysVol', 'SpecVar'])
    #put in If Else statement for conditions where the weights is 0 which will give error
    it = 0
    for i in Dates:
        for t in range(len(Industry_codes)):
            it = it + 1
            print("iteration:",it)
            print("Date:",i)
            
            Subsector = (tbl_Industry_Classification_Benchmark.loc[tbl_Industry_Classification_Benchmark["Industry Code"] == Industry_codes[t]]).loc[:,"Sub-Sector Code"].unique()
            #Select index code - Provided by the user

            #run through first function
            Results_1 = GetICsAndWeights(i,IndexCode_temp,tbl_Index_Constituents)
            Results_1_filtered = Results_1.loc[Results_1["ICB Sub-Sector"].isin(Subsector)] 

            #Place if-Else statement here:
            if len(Results_1_filtered)>0:

                Filtered_GMC = np.array(Results_1_filtered.loc[:,"Gross Market Capitalisation"])
                Total_GMC = np.array(Results_1.loc[:,"Gross Market Capitalisation"])

                Industry_Weights = pd.DataFrame((Filtered_GMC/np.sum(Total_GMC)))
                Industry_Weight_tot = "{:.2%}".format(float(np.sum(Industry_Weights)))

                ICs_temp = Results_1_filtered["Alpha"]
                mktIndexCode_temp = "J203"
                Results_2 = GetBetasMktAndSpecVols(i,ICs_temp,tbl_BA_Beta_Output,mktIndexCode_temp)

                mktVol_temp = Results_2["mktVol"][0]
                betas_temp = np.transpose(np.matrix(Results_2["Betas"]))
                specVols_temp = np.transpose(np.matrix(Results_2["specVols"]))

                Results_3 = CalcStats(Industry_Weights,betas_temp,specVols_temp,mktVol_temp)

                beta_output = Results_3["pfBeta"][0]
                SysVol_output = "{:.3%}".format(float(Results_3["pfSysVol"][0]))
                SpecVar_output = round(float(Results_3["pfSpecVol"][0]),6)
                
                date_output = pd.to_datetime(i)
                date_output = str(date_output.year)+"-Q"+str(date_output.quarter)


                df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
                print("Industry:", Industry_names[t])
            else:
                weight_output = "{:.2%}".format(0.000)
                date_output = pd.to_datetime(i)
                date_output = str(date_output.year)+"-Q"+str(date_output.quarter)
                df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': weight_output, 'Beta' : "     -", 'SysVol' : "     -", 'SpecVar' : "     -"}, ignore_index=True)
                print("Industry:", Industry_names[t])

    return df_Storage

Date_2_year = '2018'
Date_2_quarter = 4

Date_1_year = '2018'
Date_1_quarter = 1

IndexCode_temp = "TOPI"


Synthetic_Output = SyntheticTable(Date_1_year,Date_1_quarter,Date_2_year,Date_2_quarter,IndexCode_temp)


iteration: 1
Date: 2018-03-19T00:00:00.000000000
Industry: Oil & Gas
iteration: 2
Date: 2018-03-19T00:00:00.000000000
Industry: Basic Materials
iteration: 3
Date: 2018-03-19T00:00:00.000000000
Industry: Industrials
iteration: 4
Date: 2018-03-19T00:00:00.000000000
Industry: Consumer Goods
iteration: 5
Date: 2018-03-19T00:00:00.000000000
Industry: Health Care
iteration: 6
Date: 2018-03-19T00:00:00.000000000
Industry: Consumer Services
iteration: 7
Date: 2018-03-19T00:00:00.000000000
Industry: Telecommunications
iteration: 8
Date: 2018-03-19T00:00:00.000000000
Industry: Utilities
iteration: 9
Date: 2018-03-19T00:00:00.000000000


C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': weight_output, 'Beta' : "     -", 'SysVol' : "     -", 'SpecVar' : "     -"}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

Industry: Financials
iteration: 10
Date: 2018-03-19T00:00:00.000000000
Industry: Technology
iteration: 11
Date: 2018-06-18T00:00:00.000000000
Industry: Oil & Gas
iteration: 12
Date: 2018-06-18T00:00:00.000000000
Industry: Basic Materials
iteration: 13
Date: 2018-06-18T00:00:00.000000000
Industry: Industrials
iteration: 14
Date: 2018-06-18T00:00:00.000000000
Industry: Consumer Goods
iteration: 15
Date: 2018-06-18T00:00:00.000000000
Industry: Health Care
iteration: 16
Date: 2018-06-18T00:00:00.000000000
Industry: Consumer Services
iteration: 17
Date: 2018-06-18T00:00:00.000000000
Industry: Telecommunications
iteration: 18
Date: 2018-06-18T00:00:00.000000000
Industry: Utilities
iteration: 19
Date: 2018-06-18T00:00:00.000000000
Industry: Financials
iteration: 20
Date: 2018-06-18T00:00:00.000000000
Industry: Technology
iteration: 21
Date: 2018-09-25T00:00:00.000000000
Industry: Oil & Gas
iteration: 22
Date: 2018-09-25T00:00:00.000000000
Industry: Basic Materials
iteration: 23
Date: 2018-09-

C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be

Industry: Industrials
iteration: 24
Date: 2018-09-25T00:00:00.000000000
Industry: Consumer Goods
iteration: 25
Date: 2018-09-25T00:00:00.000000000
Industry: Health Care
iteration: 26
Date: 2018-09-25T00:00:00.000000000
Industry: Consumer Services
iteration: 27
Date: 2018-09-25T00:00:00.000000000
Industry: Telecommunications
iteration: 28
Date: 2018-09-25T00:00:00.000000000
Industry: Utilities
iteration: 29
Date: 2018-09-25T00:00:00.000000000
Industry: Financials
iteration: 30
Date: 2018-09-25T00:00:00.000000000
Industry: Technology
iteration: 31
Date: 2018-12-24T00:00:00.000000000
Industry: Oil & Gas
iteration: 32
Date: 2018-12-24T00:00:00.000000000
Industry: Basic Materials
iteration: 33
Date: 2018-12-24T00:00:00.000000000
Industry: Industrials
iteration: 34
Date: 2018-12-24T00:00:00.000000000
Industry: Consumer Goods
iteration: 35
Date: 2018-12-24T00:00:00.000000000
Industry: Health Care
iteration: 36
Date: 2018-12-24T00:00:00.000000000
Industry: Consumer Services
iteration: 37
Date:

C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
C:\Users\Abdud Daiyaan\AppData\Local\Temp\ipykernel_17260\1131910025.py:61: FutureWarning: The frame.append method is deprecated and will be